In [2]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [3]:
import addict
import copy
import datetime
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload

import celeri
celeri = reload(celeri)

In [4]:
import celeri
celeri = reload(celeri)

RUN_NAME = datetime.datetime.now().strftime("%y%m%d%H%M%S") + os.sep
# command_file_name = './data/western_north_america/western_north_america_command.json'
command_file_name = './data/global/global_command.json'
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure = celeri.assign_block_labels(segment, station, block, mogi, sar)
assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar)
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)


Isolating slip rate constraints
- Tensile-slip rate constraint on 11ba: rate = 0.00 (mm/yr), 1-sigma uncertainty = +/-1.00 (mm/yr)
- Tensile-slip rate constraint on 11bb: rate = 0.00 (mm/yr), 1-sigma uncertainty = +/-1.00 (mm/yr)
- Dip-slip rate constraint on AN_Iran_fault_2: rate = 0.00 (mm/yr), 1-sigma uncertainty = +/-3.00 (mm/yr)
- Dip-slip rate constraint on AN_Iran_fault_6: rate = 1.50 (mm/yr), 1-sigma uncertainty = +/-1.00 (mm/yr)
- Tensile-slip rate constraint on AWAT_fault_3: rate = 0.00 (mm/yr), 1-sigma uncertainty = +/-5.00 (mm/yr)
- Strike-slip rate constraint on Akar_connect2: rate = -8.00 (mm/yr), 1-sigma uncertainty = +/-5.00 (mm/yr)
- Dip-slip rate constraint on BOOB_fault_1: rate = 0.00 (mm/yr), 1-sigma uncertainty = +/-10.00 (mm/yr)
- Dip-slip rate constraint on BOOB_fault_4: rate = 0.00 (mm/yr), 1-sigma uncertainty = +/-10.00 (mm/yr)
- Tensile-slip rate constraint on BTFZ_Baldwin12b: rate = 0.00 (mm/yr), 1-sigma uncertainty = +/-10.00 (mm/yr)
- Tensile-slip rate cons

In [5]:
command = addict.Dict(command)